<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       HoltWintersForecaster Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>HoltWintersForecaster</b></p>
<p style = 'font-size:16px;font-family:Arial'>HoltWintersForecaster is a time series forecasting method that uses a combination of level, trend, and seasonality components to make predictions. This method is exponential smoothing method, and is an extension of simple exponential smoothing. The function uses triple exponential smoothing on a forecast model with seasonal data or double exponential smoothing for non-seasonal data, depending on the parameters.</p>

<p style = 'font-size:16px;font-family:Arial'>The seasonal aspect means the data is repetitive over some period. The level, trend, and seasonality components are used for the calculations. The level and seasonality components are additive or multiplicative. The trend remains additive.</p>

<li style = 'font-size:16px;font-family:Arial'><code>Level:</code> The average value of the time series.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Trend:</code> The rate of change of the time series.</li>
    <li style = 'font-size:16px;font-family:Arial'><code>Seasonality:</code> The periodic fluctuations or seasonal patterns in the time series.</li>

<p style = 'font-size:16px;font-family:Arial'>The alpha, beta, and gamma parameters of the function coincide with the level, trend, and seasonality, respectively, to control the main smoothing components. When values are not supplied for these parameters, their values are calculated, and available in the secondary result set.</p>

<p style = 'font-size:16px;font-family:Arial'>Exponential smoothing is a weighted average of past observations, where the weights decrease exponentially as the observations get older. The weights are determined by the smoothing parameters level, trend, and seasonality, which are values between 0 and 1.</p>

<p style = 'font-size:16px;font-family:Arial'>Once the components are estimated, HoltWintersForecaster uses them to forecast future values of the time series. The method is useful for time series data that exhibits seasonality and trend patterns. It can provide accurate forecasts even when the data is noisy or has missing values.</p>

<p style = 'font-size:16px;font-family:Arial'>HoltWintersForecaster is used in demand forecasting, financial forecasting, and inventory management.</p>



<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    HoltWintersForecaster,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_HoltWintersForecaster.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["us_air_pass"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("us_air_pass")
data

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. HoltWintersForecaster</b>
<p style = 'font-size:16px;font-family:Arial'>The The HoltWintersForecaster() function uses triple exponential smoothing on a forecast model with seasonal data. Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(HoltWintersForecaster)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the DTW function.</p>

In [ ]:
data_series_df = TDSeries(data=data,
                              row_index="idx",
                              row_index_style="SEQUENCE",
                              id="id",
                              payload_field="international",
                              payload_content="REAL")

<p style = 'font-size:16px;font-family:Arial'>We will generate forecast for 12 periods using multiplicative model.</p>

<p style = 'font-size:16px;font-family:Arial'>Input parameters for the HoltWintersForecaster function:</p>
<li style = 'font-size:16px;font-family:Arial'><code>data1:</code> Specifies the time series to forecast using historical data with series content type as 'REAL' or 'MULTIVAR_REAL'.</li>
<li style = 'font-size:16px;font-family:Arial'><code>forecast_periods:</code> Specifies the number of periods to forecast. Value must be greater than or equal to 1.</li>
<li style = 'font-size:16px;font-family:Arial'><code>model:</code> Specifies the type of Holt Winters forecasting. Permitted Values: <b>ADDITIVE</b> and <b>MULTIPLICATIVE</b></li>
<li style = 'font-size:16px;font-family:Arial'><code>fit_metrics:</code> pecifies whether to generate the result set that contains the model metadata statistics. When set to True, function generates the model statistics, otherwise not. The generated model statistics can be retrieved using the attribute "fitmetadata" of the function output.</li>
<li style = 'font-size:16px;font-family:Arial'><code>selection_metrics:</code> Specifies whether to generate the result set that contains the selection metrics. When set to True, function generates the selection metrics, otherwise not. The generated selection metrics can be retrieved using the attribute "selmetrics" of the function output.</li>

In [ ]:
uaf_out = HoltWintersForecaster(data=data_series_df,
                                    forecast_periods=12,
                                    model="MULTIPLICATIVE",
                                    residuals=True,
                                    fit_metrics=True,
                                    selection_metrics=True)

hwf_df = uaf_out.result
hwf_df

<p style = 'font-size:16px;font-family:Arial'><code>fitmetadata</code> - Available when "fit_metrics" is set to True, otherwise not</p>

In [ ]:
# Print the model statistics result.
uaf_out.fitmetadata

<p style = 'font-size:16px;font-family:Arial'><code>selmetrics</code> - Available when "selection_metrics" is set to True, otherwise not.</p>

In [ ]:
# Print the selection metrics result.
uaf_out.selmetrics 

<p style = 'font-size:16px;font-family:Arial'><code>fitresiduals </code> - Available when "residuals" is set to True, otherwise not.</p>

In [ ]:
# Print the residuals statistics result.
uaf_out.fitresiduals

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("us_air_pass")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>HoltWintersForecaster function reference: <a href = 'https://docs.teradata.com/search/all?query=HoltWintersForecaster&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>